In [1]:
## The raw data file is NOT available in the repository, only the cleaned versions are
## you can get the SBA loans dataset from kaggle or https://data.sba.gov/dataset/7-a-504-foia
import pandas as pd
fp = "../data/sba/loan_data_7a_2010_to_present.csv"
df = pd.read_csv(fp)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_8996/1504745452.py:3: DtypeWarning: Columns (12,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)


In [2]:
df.head()

,index,AsOfDate,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankStreet,...,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported
0,0,20200930,7A,"CRESA PARTNERS - DENVER, INC.",7979 E TUFTS AVE PKWY STE 810,DENVER,CO,80237,"Zions Bank, A Division of",1 S Main St,...,CO,COLORADO DISTRICT OFFICE,1.0,CORPORATION,PIF,10-31-16,NaN,0,1,35
1,1,20200930,7A,The Hilltop Tavern,4757 Folsom Blvd,Sacramento,CA,95819,Plumas Bank,336 W Main St,...,CA,SACRAMENTO DISTRICT OFFICE,6.0,CORPORATION,PIF,03-31-19,NaN,0,0,6
2,2,20200930,7A,River City Car Wash LLC,649 Harbor Blvd,West Sacramento,CA,95691,"Wells Fargo Bank, National Association",101 N Philips Ave,...,CA,SACRAMENTO DISTRICT OFFICE,3.0,CORPORATION,CHGOFF,NaN,08-24-15,320098,0,27
3,3,20200930,7A,Alphagraphics,71 Newtown Road.,Danbury,CT,6810,Union Savings Bank,226 Main St,...,CT,CONNECTICUT DISTRICT OFFICE,5.0,CORPORATION,EXEMPT,NaN,NaN,0,1,5
4,4,20200930,7A,ON SITE AUTOMOTIVE APPEARANCE,603 WOODBRIDGE COURT,MIDDLEBURY,IN,46540,"VelocitySBA, LLC",9385 Haven Avenue,...,IN,INDIANA DISTRICT OFFICE,3.0,INDIVIDUAL,PIF,02-28-13,NaN,0,0,2


In [3]:
def is_real_estate_business(x):
    x = str(x)
    if x.startswith("53"):
        return True
    else:
        return False
mask_real_estate = df["NaicsCode"].apply(is_real_estate_business)

In [4]:
df_re = df[mask_real_estate]

In [5]:
df_re["NaicsDescription"]

0                 Offices of Real Estate Agents and Brokers
34        Construction, Mining, and Forestry Machinery a...
44                  Other Activities Related to Real Estate
199       Lessors of Nonresidential Buildings (except Mi...
264       Lessors of Nonresidential Buildings (except Mi...
                                ...                        
588001     Lessors of Miniwarehouses and Self-Storage Units
588020     Lessors of Miniwarehouses and Self-Storage Units
588023    Lessors of Nonresidential Buildings (except Mi...
588025     Lessors of Miniwarehouses and Self-Storage Units
588031            Offices of Real Estate Agents and Brokers
Name: NaicsDescription, Length: 13715, dtype: object

In [6]:
disb_mask = df_re["LoanStatus"] == "COMMIT"
exempt_mask = df_re["LoanStatus"] == "EXEMPT"
drop_mask = ~(disb_mask | exempt_mask)
df_re = df_re[drop_mask].copy()
df_re = df_re.reset_index(drop=True)
del df

In [7]:
lsgb = df_re.groupby("LoanStatus")
lsgb["LoanStatus"].count()

LoanStatus
CANCLD    1667
CHGOFF     275
PIF       5065
Name: LoanStatus, dtype: int64

In [8]:
cols_to_drop = ["index", "Program"]
cols = [c for c in df_re.columns if c not in cols_to_drop]

In [9]:
df_re = df_re[cols]

In [10]:
df_re["BusinessType"].value_counts()

BusinessType
CORPORATION    6278
INDIVIDUAL      566
PARTNERSHIP     163
Name: count, dtype: int64

In [11]:
df_re.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7007 entries, 0 to 7006
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   AsOfDate               7007 non-null   int64  
 1   BorrName               7007 non-null   object 
 2   BorrStreet             7007 non-null   object 
 3   BorrCity               7007 non-null   object 
 4   BorrState              7007 non-null   object 
 5   BorrZip                7007 non-null   int64  
 6   BankName               7007 non-null   object 
 7   BankStreet             7007 non-null   object 
 8   BankCity               7007 non-null   object 
 9   BankState              7007 non-null   object 
 10  BankZip                7007 non-null   object 
 11  GrossApproval          7007 non-null   float64
 12  SBAGuaranteedApproval  7007 non-null   float64
 13  ApprovalDate           7007 non-null   object 
 14  ApprovalFiscalYear     7007 non-null   int64  
 15  Firs

In [12]:
df_re

,AsOfDate,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankStreet,BankCity,BankState,...,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported
0,20200930,"CRESA PARTNERS - DENVER, INC.",7979 E TUFTS AVE PKWY STE 810,DENVER,CO,80237,"Zions Bank, A Division of",1 S Main St,SALT LAKE CITY,UT,...,CO,COLORADO DISTRICT OFFICE,1.0,CORPORATION,PIF,10-31-16,NaN,0,1,35
1,20200930,AEG Properties,109 Walnut St,MONTICELLO,MN,55362,RiverWood Bank,14633 Edgewood Dr,BAXTER,MN,...,MN,MINNESOTA DISTRICT OFFICE,6.0,CORPORATION,PIF,02-28-17,NaN,0,0,1
2,20200930,"AP2 INVESTMENT, LTD. and ABC","1598 South Valley Parkway, Sui",Lewisville,TX,75067,Gulf Coast Bank and Trust Company,200 St. Charles Ave,NEW ORLEANS,LA,...,TX,DALLAS / FT WORTH DISTRICT OFFICE,26.0,PARTNERSHIP,CANCLD,NaN,NaN,0,0,0
3,20200930,"Morrow Enterprises, LLC",1140 Highland Avenue #184.,Manhattan beach,CA,90266,Pacific Premier Bank,"17901 Von Karman Ave, Ste 1200",IRVINE,CA,...,CA,LOS ANGELES DISTRICT OFFICE,33.0,CORPORATION,PIF,02-28-13,NaN,0,0,4
4,20200930,Kinetic Lighting Inc,7672 N Clyborn Avenue.,Sun valley,CA,91352,Bank of the Sierra,90 N Main St,PORTERVILLE,CA,...,CA,LOS ANGELES DISTRICT OFFICE,28.0,CORPORATION,PIF,12-31-12,NaN,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7002,20200930,"Landmark Escrow, Inc",9535 RESEDA BLVD Suite 100 &,NORTHRIDGE,CA,91324,East West Bank,"135 N Los Robles Ave, 7th Fl",PASADENA,CA,...,CA,LOS ANGELES DISTRICT OFFICE,30.0,CORPORATION,CANCLD,NaN,NaN,0,0,27
7003,20200930,"Property Valuation Advisors, I",850 W JACKSON BLVD,CHICAGO,IL,60607,"Newtek Small Business Finance, Inc.",1981 Marcus Avenue,LAKE SUCCESS,NY,...,IL,ILLINOIS DISTRICT OFFICE,7.0,CORPORATION,CANCLD,NaN,NaN,0,0,7
7004,20200930,"Proxy Home Services, Inc",174 BROOKWOOD RD,ASHEVILLE,NC,28804,"Mountain Bizcapital, Inc.",153 South Lexington Avenue,Asheville,NC,...,NC,NORTH CAROLINA DISTRICT OFFICE,11.0,CORPORATION,CANCLD,NaN,NaN,0,0,5
7005,20200930,"AVANT GARDE REE2, LLC",496 MCBRIDE LANE,SEVERNA PARK,MD,21146,Truist Bank d/b/a Branch Banking & Trust Co,214 N Tryon St,CHARLOTTE,NC,...,MD,BALTIMORE DISTRICT OFFICE,4.0,CORPORATION,CANCLD,NaN,NaN,0,0,25


In [13]:
def earnings_from_loan(row):
    SERVICE_CHARGE_PER_TERM = 50
    term_in_years = row["TermInMonths"]/12
    principal = row["GrossApproval"] 
    rate = row["InitialInterestRate"]/100
    earnings = (principal * term_in_years * rate) - (SERVICE_CHARGE_PER_TERM*12 *term_in_years) 
    return earnings

df_re["cum_earnings"] = df_re.apply(earnings_from_loan, axis=1)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_8996/3449569002.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_re["cum_earnings"] = df_re.apply(earnings_from_loan, axis=1)


In [14]:
df_re.apply(earnings_from_loan, axis=1)

0         87850.0
1        168000.0
2       2070300.0
3       1335000.0
4        296400.0
          ...    
7002     781500.0
7003     114000.0
7004      99000.0
7005    1282125.0
7006      99120.0
Length: 7007, dtype: float64

In [15]:
calc_cols = ["TermInMonths", "GrossApproval", "InitialInterestRate", "cum_earnings"]
df_re[calc_cols]

,TermInMonths,GrossApproval,InitialInterestRate,cum_earnings
0,84,250000.0,5.26,87850.0
1,240,150000.0,6.00,168000.0
2,309,1350000.0,6.00,2070300.0
3,300,900000.0,6.00,1335000.0
4,120,504000.0,6.00,296400.0
...,...,...,...,...
7002,120,1500000.0,5.25,781500.0
7003,120,200000.0,6.00,114000.0
7004,120,200000.0,5.25,99000.0
7005,300,1153000.0,4.50,1282125.0


In [16]:
df_re["GrossApproval"].describe()

count    7.007000e+03
mean     5.129295e+05
std      8.228918e+05
min      3.200000e+03
25%      5.000000e+04
50%      1.700000e+05
75%      5.983000e+05
max      5.000000e+06
Name: GrossApproval, dtype: float64

In [17]:
IQR = df_re["GrossApproval"].describe()["75%"] - df_re["GrossApproval"].describe()["25%"]
threshold = 1.5 * IQR +  df_re["GrossApproval"].describe()["75%"] 
inlier_loans = df_re["GrossApproval"] <= threshold
df_re = df_re[inlier_loans]

In [18]:
df_re["cum_earnings"].mean()

266977.95159741375

In [19]:
bad_loans = (df_re["LoanStatus"] == "CANCLD" ) | (df_re["LoanStatus"] == "CHGOFF")
df_bad_loans = df_re[bad_loans].copy()
bad_loan_data_cols = ["GrossApproval",  "GrossChargeOffAmount", "LoanStatus"]
df_bad_loans[bad_loan_data_cols]

,GrossApproval,GrossChargeOffAmount,LoanStatus
2,1350000.0,0,CANCLD
7,503000.0,417840,CHGOFF
20,385000.0,0,CANCLD
22,5000.0,306,CHGOFF
37,202500.0,0,CANCLD
...,...,...,...
7001,1000000.0,0,CANCLD
7003,200000.0,0,CANCLD
7004,200000.0,0,CANCLD
7005,1153000.0,0,CANCLD


In [20]:
df_bad_loans["GrossApproval"].mean()

280291.0111241218

In [21]:
from datetime import datetime
def convert_to_date(x):
    x = str(x)
    Y = x[:4]
    m = x[4:6]
    d = x[6:]
    SEP = "-"
    ds = Y + SEP + m + SEP + d
    cd = datetime.strptime(ds, "%Y-%m-%d")
    return cd 
df_re["NAsOfDate"] = df_re["AsOfDate"].apply(convert_to_date)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_8996/156345604.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_re["NAsOfDate"] = df_re["AsOfDate"].apply(convert_to_date)


In [22]:
df_re["NAsOfDate"]

0      2020-09-30
1      2020-09-30
2      2020-09-30
3      2020-09-30
4      2020-09-30
          ...    
7001   2020-09-30
7003   2020-09-30
7004   2020-09-30
7005   2020-09-30
7006   2020-09-30
Name: NAsOfDate, Length: 6251, dtype: datetime64[ns]

In [23]:
df_re = df_re.drop(columns=["AsOfDate"])
df_re = df_re.rename({"NAsOfDate": "AsOfDate"}, axis = 1)

In [24]:
df_re.columns

Index(['BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip',
       'BankName', 'BankStreet', 'BankCity', 'BankState', 'BankZip',
       'GrossApproval', 'SBAGuaranteedApproval', 'ApprovalDate',
       'ApprovalFiscalYear', 'FirstDisbursementDate', 'DeliveryMethod',
       'subpgmdesc', 'InitialInterestRate', 'TermInMonths', 'NaicsCode',
       'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ProjectCounty',
       'ProjectState', 'SBADistrictOffice', 'CongressionalDistrict',
       'BusinessType', 'LoanStatus', 'PaidInFullDate', 'ChargeOffDate',
       'GrossChargeOffAmount', 'RevolverStatus', 'JobsSupported',
       'cum_earnings', 'AsOfDate'],
      dtype='object')

In [25]:
df_re

,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankStreet,BankCity,BankState,BankZip,...,CongressionalDistrict,BusinessType,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,cum_earnings,AsOfDate
0,"CRESA PARTNERS - DENVER, INC.",7979 E TUFTS AVE PKWY STE 810,DENVER,CO,80237,"Zions Bank, A Division of",1 S Main St,SALT LAKE CITY,UT,84133,...,1.0,CORPORATION,PIF,10-31-16,NaN,0,1,35,87850.0,2020-09-30
1,AEG Properties,109 Walnut St,MONTICELLO,MN,55362,RiverWood Bank,14633 Edgewood Dr,BAXTER,MN,56425,...,6.0,CORPORATION,PIF,02-28-17,NaN,0,0,1,168000.0,2020-09-30
2,"AP2 INVESTMENT, LTD. and ABC","1598 South Valley Parkway, Sui",Lewisville,TX,75067,Gulf Coast Bank and Trust Company,200 St. Charles Ave,NEW ORLEANS,LA,70130,...,26.0,PARTNERSHIP,CANCLD,NaN,NaN,0,0,0,2070300.0,2020-09-30
3,"Morrow Enterprises, LLC",1140 Highland Avenue #184.,Manhattan beach,CA,90266,Pacific Premier Bank,"17901 Von Karman Ave, Ste 1200",IRVINE,CA,92614,...,33.0,CORPORATION,PIF,02-28-13,NaN,0,0,4,1335000.0,2020-09-30
4,Kinetic Lighting Inc,7672 N Clyborn Avenue.,Sun valley,CA,91352,Bank of the Sierra,90 N Main St,PORTERVILLE,CA,93257,...,28.0,CORPORATION,PIF,12-31-12,NaN,0,0,4,296400.0,2020-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7001,"White Duck Realty, LLC",Deham Mall 300 Providence Hw,Dedham,MA,2026,Camden National Bank,2 Elm St,CAMDEN,ME,4843.0,...,8.0,CORPORATION,CANCLD,NaN,NaN,0,0,25,344000.0,2020-09-30
7003,"Property Valuation Advisors, I",850 W JACKSON BLVD,CHICAGO,IL,60607,"Newtek Small Business Finance, Inc.",1981 Marcus Avenue,LAKE SUCCESS,NY,11042.0,...,7.0,CORPORATION,CANCLD,NaN,NaN,0,0,7,114000.0,2020-09-30
7004,"Proxy Home Services, Inc",174 BROOKWOOD RD,ASHEVILLE,NC,28804,"Mountain Bizcapital, Inc.",153 South Lexington Avenue,Asheville,NC,28801.0,...,11.0,CORPORATION,CANCLD,NaN,NaN,0,0,5,99000.0,2020-09-30
7005,"AVANT GARDE REE2, LLC",496 MCBRIDE LANE,SEVERNA PARK,MD,21146,Truist Bank d/b/a Branch Banking & Trust Co,214 N Tryon St,CHARLOTTE,NC,28202.0,...,4.0,CORPORATION,CANCLD,NaN,NaN,0,0,25,1282125.0,2020-09-30


In [26]:
df_re.shape

(6251, 36)

In [27]:
float(5065/7007)

0.7228485799914371

In [28]:
keep = ["BorrName", "BorrZip", "BankName", "BankZip", "GrossApproval", "SBAGuaranteedApproval",\
        "InitialInterestRate", "TermInMonths", "ProjectState", "SBADistrictOffice",\
        "CongressionalDistrict", "BusinessType","RevolverStatus", "JobsSupported", "LoanStatus"]
        

In [29]:
df_re = df_re[keep]
fp = "../data/sba_real_estate.csv"
df_re.to_csv(fp, index=False)

In [34]:
bzgb = df_re.groupby(["BorrZip"])

In [37]:
df_re["LoanStatus"].value_counts()

LoanStatus
PIF       4543
CANCLD    1444
CHGOFF     264
Name: count, dtype: int64

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_8996/1886316936.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_re["D_BorrZip"] = df_re["BorrZip"].apply(bad_borr_zip)


,BorrName,BorrZip,BankName,BankZip,GrossApproval,SBAGuaranteedApproval,InitialInterestRate,TermInMonths,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,RevolverStatus,JobsSupported,LoanStatus
0,"CRESA PARTNERS - DENVER, INC.",80237,"Zions Bank, A Division of",84133,250000.0,125000.0,5.26,84,CO,COLORADO DISTRICT OFFICE,1.0,CORPORATION,1,35,PIF
1,AEG Properties,55362,RiverWood Bank,56425,150000.0,135000.0,6.00,240,MN,MINNESOTA DISTRICT OFFICE,6.0,CORPORATION,0,1,PIF
4,Kinetic Lighting Inc,91352,Bank of the Sierra,93257,504000.0,453600.0,6.00,120,CA,LOS ANGELES DISTRICT OFFICE,28.0,CORPORATION,0,4,PIF
5,St. John Video,46373,BMO Harris Bank National Association,60603,50000.0,25000.0,5.75,24,IN,INDIANA DISTRICT OFFICE,1.0,INDIVIDUAL,1,0,PIF
6,Victory Enterprises Inc,87701,Century Bank,87501,70000.0,35000.0,6.00,128,NM,NEW MEXICO DISTRICT OFFICE,3.0,CORPORATION,0,1,PIF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884,Legacy Warehousing LLC,54937,National Exchange Bank and Trust,54935,180000.0,135000.0,6.75,240,WI,WISCONSIN DISTRICT OFFICE,6.0,CORPORATION,0,0,PIF
6902,Makkar Enterprise LLC,91739,"U.S. Bank, National Association",45202,50000.0,25000.0,10.50,84,CA,SANTA ANA DISTRICT OFFICE,31.0,CORPORATION,1,1,PIF
6926,"Sand Highway RV Rental, LLC",85373,Citizens Bank,37643,597600.0,448200.0,7.50,120,AZ,ARIZONA DISTRICT OFFICE,8.0,CORPORATION,0,5,PIF
6930,"Centera Land & Title Services,",92123,"U.S. Bank, National Association",45202,200000.0,100000.0,7.75,84,CA,SAN DIEGO DISTRICT OFFICE,52.0,CORPORATION,1,8,PIF
